In [26]:
import pandas as pd
from pandas_profiling import ProfileReport
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

from sklearn.preprocessing import StandardScaler

from statsmodels.stats.outliers_influence import variance_inflation_factor

# Import dataset

In [2]:
turbine = 'T11'
path_dataset = f'../dataset/df_turbine_{turbine}.json'

## dataset

In [3]:
df = pd.read_json(path_dataset)
df.set_index('timestamp', inplace=True)
df.sort_index(ascending=True,inplace=True)
df.head()

,turbine_id,grd_prod_pwr_min,gen_rpm_avg,blds_pitchangle_max,grd_prod_pwr_avg,amb_winddir_abs_avg,gen_bear_temp_avg,amb_windspeed_min,grd_prod_reactpwr_max,grd_prod_curphse1_avg,...,grd_prod_voltphse1_avg,grd_prod_psblepwr_std,grd_prod_cosphi_avg,prod_latestavg_actpwrgen0,prod_latestavg_actpwrgen1,prod_latestavg_actpwrgen2,rtr_rpm_std,component,remarks,failure
timestamp,,,,,,,,,,,,,,,,,,,,,
2016-01-01 00:00:00,T11,154.1,1270.9,0.9,246.1,199.1,42.0,2.6,-72.5,197.0,...,400.7,51.7,0.9,0.0,41059.0,0.0,0.2,None,None,0
2016-01-01 00:10:00,T11,191.3,1303.6,-0.8,304.7,207.5,43.0,1.7,-76.8,245.0,...,400.4,54.8,1.0,0.0,50808.0,0.0,0.4,None,None,0
2016-01-01 00:20:00,T11,211.3,1344.3,-1.3,364.2,190.5,43.0,1.5,-33.7,292.3,...,399.8,50.2,1.0,0.0,60667.0,0.0,0.2,None,None,0
2016-01-01 00:30:00,T11,316.5,1377.5,0.3,465.0,214.6,43.0,2.7,-88.7,374.6,...,399.6,60.1,1.0,0.0,77486.0,0.0,0.4,None,None,0
2016-01-01 00:40:00,T11,204.3,1396.2,-1.0,404.6,212.1,43.0,1.7,-91.1,327.7,...,398.8,99.0,1.0,0.0,67378.0,0.0,0.6,None,None,0


## check number of failures

In [4]:
df.loc[~df['component'].isnull(),['turbine_id','component', 'remarks','failure']]


,turbine_id,component,remarks,failure
timestamp,,,,
2016-03-03 19:00:00,T11,GENERATOR,Electric circuit error in generator,1
2016-10-17 17:44:00,T11,HYDRAULIC_GROUP,Hydraulic group error in the brake circuit,1
2017-04-26 18:06:00,T11,HYDRAULIC_GROUP,Hydraulic group error in the brake circuit,1


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 87251 entries, 2016-01-01 00:00:00 to 2017-09-01 00:00:00
Data columns (total 85 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   turbine_id                   87251 non-null  object 
 1   grd_prod_pwr_min             87248 non-null  float64
 2   gen_rpm_avg                  87248 non-null  float64
 3   blds_pitchangle_max          87248 non-null  float64
 4   grd_prod_pwr_avg             87248 non-null  float64
 5   amb_winddir_abs_avg          87248 non-null  float64
 6   gen_bear_temp_avg            87247 non-null  float64
 7   amb_windspeed_min            87248 non-null  float64
 8   grd_prod_reactpwr_max        87248 non-null  float64
 9   grd_prod_curphse1_avg        87248 non-null  float64
 10  grd_prod_pwr_max             87248 non-null  float64
 11  grd_prod_pwr_std             87248 non-null  float64
 12  blds_pitchangle_min          87248 non-

In [6]:
datatypes = pd.DataFrame(df.dtypes, columns=['dtypes'])
datatypes['dtypes'].value_counts()

float64    81
object      3
int64       1
Name: dtypes, dtype: int64

In [7]:
datatypes[datatypes['dtypes'] == 'int64']

,dtypes
failure,int64


There is only 3 columns as string

In [8]:
tags_float = datatypes[datatypes['dtypes'] == 'float'].index.tolist()
tags_str = datatypes[datatypes['dtypes'] == 'object'].index.tolist()

## Splitting Training and Test Set

In [9]:
training_set = df.loc['2016-01-01':'2016-12-31'].copy()
test_set =  df.loc['2017-01-01':'2017-12-31'].copy()

# Analyse Training Set

There is many many columns to analyse, so in order to make it simple and reduce the dataset 

## Check for Missing Data

In [10]:
pct_null = df.isnull().sum() /df.shape[0]
pct_null.sort_values(ascending=False)


component                  0.999966
remarks                    0.999966
gen_bear_temp_avg          0.000046
grd_prod_cosphi_avg        0.000046
rtr_rpm_std                0.000034
                             ...   
grd_busbar_temp_avg        0.000034
grd_prod_psblepwr_min      0.000034
hvtrafo_phase2_temp_avg    0.000034
turbine_id                 0.000000
failure                    0.000000
Length: 85, dtype: float64

The columns don't have high percentage of null data

## Check for Inf values

In [11]:
# check for infinite values
d = ~np.isfinite(training_set[tags_float])
d.sum()

grd_prod_pwr_min             2
gen_rpm_avg                  2
blds_pitchangle_max          2
grd_prod_pwr_avg             2
amb_winddir_abs_avg          2
                            ..
grd_prod_cosphi_avg          2
prod_latestavg_actpwrgen0    2
prod_latestavg_actpwrgen1    2
prod_latestavg_actpwrgen2    2
rtr_rpm_std                  2
Length: 81, dtype: int64

In [12]:
# replace inf values
training_set[tags_float] = training_set[tags_float].replace([np.inf, -np.inf], np.nan)

## Fillna

In [18]:
# fillna
training_set[tags_float] = training_set[tags_float].fillna(method='backfill')

In [19]:
pct_null = training_set[tags_float].isnull().sum() /training_set[tags_float].shape[0]
pct_null.sort_values(ascending=False)


grd_prod_pwr_min          0.0
nac_temp_avg              0.0
gen_bear2_temp_avg        0.0
gen_rpm_max               0.0
amb_windspeed_est_avg     0.0
                         ... 
amb_windspeed_avg         0.0
grd_prod_voltphse3_avg    0.0
grd_busbar_temp_avg       0.0
grd_prod_psblepwr_min     0.0
rtr_rpm_std               0.0
Length: 81, dtype: float64

## Check for tags with high correlation and drop one of them - VIF

In [86]:
# Scaling first to ease the calculations
scaler = StandardScaler()

X = training_set[tags_float[:30]].copy()
X_scaled = scaler.fit_transform(X)

# Keep DataFrame indexes ---------------------------------------------------------
X_scaled = pd.DataFrame(data=X_scaled, columns=X.columns, index=X.index)

vif_factor_threshold = 5
iter = 1
span_tags_to_remove = 2

while iter > 0:
    print(f'\r{len(X_scaled.columns)}', end="")

    vif = pd.DataFrame()
    vif["features"] = X_scaled.columns
    vif["VIF Factor"] = [variance_inflation_factor(X_scaled.values, i) for i in range(X_scaled.shape[1])]
    vif_high = vif.sort_values(by=['VIF Factor'], ascending=False)

    if vif_high[vif_high['VIF Factor'] > vif_factor_threshold].shape[0] == 0:
        iter = 0
    else:
        tag_name = vif_high[vif_high['VIF Factor'] > vif_factor_threshold].iloc[:span_tags_to_remove]['features']
        cols = X_scaled.columns.tolist()
        for t in tag_name:
            cols.remove(t)
        X_scaled = X_scaled[cols].copy()

display(vif_high)

11

,features,VIF Factor
7,grd_prod_psblepwr_min,3.680249
10,amb_windspeed_std,3.313173
2,amb_windspeed_min,2.569175
4,grd_prod_pwr_std,2.410613
9,grd_prod_psblecap_min,2.350964
5,blds_pitchangle_min,2.182718
3,grd_prod_reactpwr_max,2.054615
0,blds_pitchangle_max,1.956916
8,grd_prod_voltphse3_avg,1.744336
1,amb_winddir_abs_avg,1.177370


### Check for low variance 

In [82]:
pd.set_option('display.float_format', lambda x: '%.3f' % x)
df.std(numeric_only=True).sort_values(ascending=True).head(20)

prod_latestavg_actpwrgen2     0.000
prod_latestavg_reactpwrgen2   0.000
failure                       0.006
grd_prod_freq_avg             0.012
grd_prod_cosphi_avg           0.174
amb_windspeed_std             0.809
rtr_rpm_std                   0.990
amb_windspeed_min             1.024
grd_prod_voltphse1_avg        1.477
grd_prod_voltphse2_avg        1.513
grd_prod_voltphse3_avg        2.113
amb_windspeed_est_avg         3.271
amb_windspeed_avg             3.319
blds_pitchangle_std           4.129
cont_vcp_wtrtemp_avg          4.947
nac_temp_avg                  5.015
rtr_rpm_avg                   5.226
rtr_rpm_min                   5.268
cont_top_temp_avg             5.289
cont_vcp_temp_avg             5.350
dtype: float64

In [38]:
sns.heatmap(training_set.isnull(), cbar=False)

<AxesSubplot:ylabel='timestamp'>

In [37]:
profile = ProfileReport(training_set, title="Pandas Profiling Report", explorative=True)
profile.to_file(f'../output_EDA/{turbine}_pandas_profiling_training.html')
print('ok')

Summarize dataset:  93%|█████████▎| 90/97 [00:38<00:15,  2.18s/it, Calculate cramers correlation]C:\Users\rhamacherhay\Anaconda3\envs\hamacher_env\lib\site-packages\pandas_profiling\model\correlations.py:61: UserWarning: There was an attempt to calculate the cramers correlation, but this failed.
To hide this warning, disable the calculation
(using `df.profile_report(correlations={"cramers": {"calculate": False}})`
If this is problematic for your use case, please report this as an issue:
https://github.com/pandas-profiling/pandas-profiling/issues
(include the error message: 'No data; `observed` has size 0.')
  (include the error message: '{error}')"""
Summarize dataset:  26%|██▌       | 1602/6181 [05:37<16:05,  4.74it/s, scatter grd_prod_psbleind_max, grd_prod_psbleind_min]


KeyboardInterrupt: 